In [1]:
import numpy as np
import signac
import hoomd
import gsd.hoomd
import random

In [2]:
project = signac.get_project("/home/stephaniemccallu/scratch/Entanglements")

In [11]:
def bootstrap_msd(job, start=0, stop=None, stride=1):
    """
    length = length of molecules
    chains = number of chains in the system
    """
    steps_per_frame = int(5e5)
    ts = job.doc.real_time_step * 1e-15
    ts_frame = steps_per_frame * ts
    rand_list = []
    msd_list = []
    n = 10
    for i in range(n):
        rand_list.append(random.randint(0,199))   
        msd = msd_from_gsd(
            gsdfile=job.fn("production3-center.gsd"),
            start=rand_list[i],
            stop=(rand_list[i]+200),
            atom_types="B",
            msd_mode="direct")
            msd_results = np.copy(msd.msd)
            msd_list.append(msd_results)
    msd_avg = np.mean(msd_list)
    conv_factor = job.doc.ref_length**2
    job.doc.msd_units = "nm**2"
    msd_avg *= conv_factor
    time_array = np.arange(0, len(msd.msd), 1) * ts_frame
    np.save(file=job.fn("msd_time_bs.npy"), arr=time_array)
    np.save(file=job.fn("msd_data_real_nm_squared_bs.npy"), arr=msd_avg)
    np.save(file=job.fn("msd_data_reduced_bs.npy"), arr=msd.msd)

In [9]:
for job in project.find_jobs():
    if job.isfile("production3-center.gsd"):
        bootstrap_msd(job)

400
/home/stephaniemccallu/scratch/Entanglements/workspace/709e873d95982e2c7d48c8ff2f7836e9
400
/home/stephaniemccallu/scratch/Entanglements/workspace/0fef6f5d6b45614c995e7714c8b14f9f
400
/home/stephaniemccallu/scratch/Entanglements/workspace/bb60a2a949607785374583f9538aaac1
400
/home/stephaniemccallu/scratch/Entanglements/workspace/89314312f609e3eb0cc905187d4092a2
400
/home/stephaniemccallu/scratch/Entanglements/workspace/7b9c1fbdb08258bc6f94869866ab8051
400
/home/stephaniemccallu/scratch/Entanglements/workspace/101f829356e885ab16664b76406d3146
400
/home/stephaniemccallu/scratch/Entanglements/workspace/cbaf2f651dbac0cfed8ba352e82b41e9
